In [ ]:
# 初始化 BOW：原始数据读入read data and save it in data['nearRaw.train'] 和 data['nearRaw.test']
if ('init' == phase):# and ('BOW' == idpart['model.word']):
    data = {}
    data['nearRaw.train'] = {'content':[], 'class':[]}
    data['nearRaw.test'] = {'content':[], 'class':[]}

    for tpart in ['train', 'test']:
        dirpath = paths['dir.{}'.format(tpart)]
        for (ind, cls) in enumerate(os.listdir(dirpath)):
            clspath = os.path.join(dirpath, cls)
            files = os.listdir(clspath)
            for f in files:
                fpath = os.path.join(clspath, f)
                with open(fpath, 'r') as readf:
                    data['nearRaw.{}'.format(tpart)]['content'].append(readf.read())
                    data['nearRaw.{}'.format(tpart)]['class'].append(cls)
        tmp = data['nearRaw.{}'.format(tpart)]
        ind = (random.sample(range(len(tmp['class'])), 1))[0]
    #     print("sample(transformed) from {}[{}]:\n[content]\n {}\n[class]\n{}".format(
    #             tpart, ind, tmp['content'][ind], tmp['class'][ind]
    #         )
    #     )
    #     print() 

    # print("Step 2 Succeed")

In [ ]:
# 建立【文档】表示

if ('modeling.document' == phase):# and ('BOW' == idpart['model.word']):
    # Step: CountVectorizer.fit + TfidfVectorizer.transform + save in Pandas.DataFrame
    # A. 使用 sklearn.feature_extraction.text.TfidfVectorizer.fit 拟合训练数据，建立 BOW+TF-IDF 
    # B. 使用 sklearn.feature_extraction.text.TfidfVectorizer.transform 将data['nearRaw.train']中的 stringContent 和 data['nearRaw.test']中的 stringContent 进行处理，
    # 将 BOW+TF-IDF 表示结果输出到 data['matrix.train'] 与 data['matrix.test'] 中，供后续学习和训练使用
    # 
    # C. 将 data['matrix.train'] 与 data['matrix.test'] 转换成 Pandas.DataFrame 格式，保存到 df['train'] 和 df['test'] 中（df 为字典格式：String -> DataFrame）
    
    # Substep A: vectorization + TF-IDF calculation
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer

    if 'NO' == idpart['useStopwords']:
        countVector = CountVectorizer(analyzer='word', max_features=vecsize)
    else:
        import nltk
        wordset = []        
        for tpart in ['dir.train', 'dir.test']:
            dirpath = paths[tpart]
            for cls in os.listdir(dirpath):
                clspath = os.path.join(dirpath, cls)
                files = os.listdir(clspath)
                for f in files:
                    fpath = os.path.join(clspath, f)
                    with open(fpath, 'r', encoding="latin-1") as fpath_f:
                        tmpFileStr = fpath_f.read()
                    tmpFileStr = tmpFileStr.split()
                    wordset += tmpFileStr

        wordDist = nltk.FreqDist(wordset)
        freqword = [i[0]for i in wordDist.most_common(30)]
        countVector = CountVectorizer(analyzer='word', max_features=vecsize, stop_words=freqword)
    countVector.fit(data['nearRaw.train']['content'])
    tfidf = TfidfTransformer()

#     print("Substep A finished.")
#     print("--------------------------------------------------")

    ## Substep B: Transformation

    for tpart in ['train', 'test']:
        data['matrix.{}'.format(tpart)] = countVector.transform(data['nearRaw.{}'.format(tpart)]['content']) 
        data['matrix.{}'.format(tpart)] = tfidf.fit_transform(data['matrix.{}'.format(tpart)])
        ind = (random.sample(range(data['matrix.{}'.format(tpart)].shape[0]), 1))[0]
#         print("sample for matrix.{}".format(tpart))
#         print("from ind: {}".format(ind))
#         print(data['matrix.{}'.format(tpart)][ind])
#         print() 

#     print("Substep B finished.")
#     print("--------------------------------------------------")

    # Substep C: integrate data into DataFrame format

#     csvpath_root = os.path.join(paths['dir.dataroot'], 'data_CSV')
#     if not os.path.isdir(csvpath_root):
#         os.mkdir(csvpath_root)

    df = {}
    for tpart in ['train', 'test']:
        datadict = {}
        datadict['class'] = data['nearRaw.{}'.format(tpart)]['class']
        for col in range(data['matrix.{}'.format(tpart)].shape[1]):
            datadict[col]= [i[0] for i in data['matrix.{}'.format(tpart)].getcol(col).toarray()]
    #         datadict[str(col)]= [i[0] for i in data['matrix.{}'.format(tpart)].getcol(col).toarray()]

        df[tpart] = pd.DataFrame(data=datadict)
        # Shuffle the dataset
        df[tpart] = df[tpart].sample(frac=1)
        df[tpart].reset_index(drop=True, inplace=True)
#         print("See df[{}]".format(tpart))
        display(df[tpart])
#         print("\n\n\n")
#         #write data in DataFrame into CSV
#         csvpath = os.path.join(csvpath_root, "{}-{}-{}.csv".format(tpart, idpart['model.word'], idpart['model.document']))
#         df[tpart].to_csv(csvpath, columns=df[tpart].columns)

#     print("Substep C finished.")
#     print("--------------------------------------------------")

#     print("Step 3 Succeed.")

    # 繁琐点：研究如何把 CSR 矩阵中的数据规整好放到 DataFrame 中，并与 Class 一一对应
    
    # Save in train
    X_train, y_train = df['train'].drop('class', axis=1), df['train']['class']
    
    # Save in test
    X_test, y_test_true = df['test'].drop('class', axis=1), df['test']['class']

In [ ]:
# if ('modeling.document' == phase) :#and  ('TF*IDF-LSA' == idpart['model.document']):
#     # Step: LSA based on BOW+TFIDF

#     from sklearn.decomposition import TruncatedSVD

#     svd = TruncatedSVD(n_components=200, n_iter=10, random_state=19)
#     svd.fit(df['train'].drop('class', axis=1))

#     df_new = {}
#     for tpart in ['train', 'test']:
#         df_new[tpart] = {}
#         datadict = {}
#         X_LSA = svd.transform(df[tpart].drop('class', axis=1))
#         for col in range(X_LSA.shape[1]):
#             datadict[col] = X_LSA[:, col]
#         df_new[tpart]['y'] = df[tpart]['class']
#         df_new[tpart]['x'] = pd.DataFrame(data=datadict)
#         df_new[tpart]['all'] = df_new[tpart]['x'].join(df_new[tpart]['y'])
#         display(df_new[tpart]['all'])

#     # Save in train
#     X_train, y_train = df_new['train']['x'], df_new['train']['y']

#     # Save in test
#     X_test, y_test_true = df_new['test']['x'], df_new['test']['y']

In [ ]:
# if ('modeling.document' == phase) and  ('TF*IDF-LDA' == idpart['model.document']):
#     # Step: LDA based on BOW+TFIDF

#     from sklearn.decomposition import LatentDirichletAllocation

#     lda = LatentDirichletAllocation(n_topics=50, max_iter=10, random_state=19, learning_method='batch')
#     lda.fit(df['train'].drop('class', axis=1))

#     df_new = {}
#     for tpart in ['train', 'test']:
#         df_new[tpart] = {}
#         datadict = {}
#         X_LDA = lda.transform(df[tpart].drop('class', axis=1))
#         for col in range(X_LDA.shape[1]):
#             datadict[col] = X_LDA[:, col]
#         df_new[tpart]['y'] = df[tpart]['class']
#         df_new[tpart]['x'] = pd.DataFrame(data=datadict)
#         df_new[tpart]['all'] = df_new[tpart]['x'].join(df_new[tpart]['y'])
#         display(df_new[tpart]['all'])
        
#     # Save in train
#     X_train, y_train = df_new['train']['x'], df_new['train']['y']

#     # Save in test
#     X_test, y_test_true = df_new['test']['x'], df_new['test']['y']